# GMM ensemble
Created by: Lea Poropat

Last edit: 2023/06/23

Unsupervised classification of the ocean based on satellite altimetry data with an ensemble of Gaussian Mixture Models (GMMs). The ensemble consists of N models, each fitted to K mixture components or classes. The classes from all ensemble members are matched, considering them the same class if the correlation between class means is higher than the correlation cutoff. Then for each grid point, we use soft voting, i.e., calculate the sum of probabilities for each class that all ensemble members provided, and the grid point is assigned the class with the highest sum of probabilities. Likelihood is calculated by dividing this highest sum of probabilities with the number of ensemble members N. The algorithm then checks if any classes are too small (smaller than the minimal size given) and removes those classes, re-assigning the grid point the next best class. There is also a possibility to only use votes with high probabilities by setting the minproba parameter.

A small subset of data is saved as a test set (~10%), while the training set is randomly drawn from the rest of it. test_size refers to the size of the test set, while train_size refers to the percentage of the whole data set randomly taken from the remaining data. E.g., test_size = 10%, train_size = 30% means that 10% is taken aside for testing, while the training set is a random third (33%) of the remaining data set.

Input data is in .npy format. The models needs 3 files: EOF maps (nEOFs x nlat x nlon), longitude grid, and latitude grid (both nlat x nlon). The results are saved in .mat format. The name of the results file is created based on model and ensemble parameters.

### <font color = "red">Parameters</font>

In [1]:
# region
reg = 'North'

# number of PCs used
nPC = 3

# file names
pth = r'Files/' + reg + '_'
fleof = pth + 'eof_maps.npy'
fllon = pth + 'Lon.npy'
fllat = pth + 'Lat.npy'

# percent of training set
test_size = 0.1
train_size = 0.9

# number of classes in the GMM
K = 6

# correlation cutoff
corrlim = 0.98

# minimal probability to take into account a vote (0 = all probabilities are taken into account)
minproba = 0.0

# minimal allowed class size (number of gridpoints in combined train and test set)
minsize = 100

# number of ensemble members
N = 200

# number of the same model
no = 1

# model name
modelname = reg + '_PCs' + str(nPC) + \
    '_tr' + str(int(train_size*100)) + \
    '_K' + str(K) + '_r' + \
    str(int(corrlim*100)).zfill(2) + \
    '_p' + str(int(minproba*100)).zfill(2) + \
    '_gp' + str(minsize) + \
    '_N' + str(N) + \
    '_no' + str(no)

# results
res = r'Files/'

### Loading the libraries

In [2]:
#import os
#import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from scipy.stats import pearsonr
import scipy.io
import time

C:\Users\xporol\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\xporol\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\xporol\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Loading and reshaping data + removing empty grid points

In [3]:
# loading the data
X0 = np.load(fleof)
Lon0 = np.load(fllon)
Lat0 = np.load(fllat)
nlat, nlon = np.shape(Lon0)

# 1D longitude and latitude
lon = Lon0[0, :]
lat = Lat0[:, 0]

# reshape to 2D
X1 = np.reshape(X0,(len(X0),-1),'C').transpose()
Lon1 = np.reshape(Lon0, -1, 'C').transpose()
Lat1 = np.reshape(Lat0, -1, 'C').transpose()

# remove land and empty grid points
ind_ocean = np.squeeze(np.nonzero(np.squeeze(~np.all(np.isnan(X1), axis = 1))))
X2 = X1[ind_ocean, :]
Lon2 = Lon1[ind_ocean]
Lat2 = Lat1[ind_ocean]

# check if there are any gaps in the dataset
gaps = np.argwhere(np.any(np.isnan(X2), axis = 1))
gaps = np.squeeze(gaps)

# printing the summary
if len(gaps)==0:
    print('Land grid points and grid points with missing data removed!')
print('Shape of the dataset after removing empty points: %s' % str(np.shape(X2)))

Land grid points and grid points with missing data removed!
Shape of the dataset after removing empty points: (2381, 300)


### Selecting the principal components that will be used

In [4]:
# select the PCs that combined make up {cutoff}% of variability
X = X2[:, :nPC]
npcs = np.shape(X)[1]

# print the number of PCs used
print('Number of PCs used: %3i' % (nPC))

Number of PCs used:   3


### Train-test set split

In [5]:
# selecting a small subset of data that will be kept separately for testing
ind = np.array(range(0, len(X)))

Xtr, Xte, Lontr, Lonte, Lattr, Latte, indtr, indte = train_test_split(X, Lon2, Lat2, ind, test_size = test_size)
ntr = np.shape(Xtr)[0]
nte = np.shape(Xte)[0]

### GMM

In [6]:
# creating the results matrices
y = np.empty((ntr+nte, N), dtype = np.int16)
proba = np.empty((ntr+nte, N))
class_means = np.empty((K, npcs, N))

for i in range(N):
    # creating the model
    gmm = GaussianMixture(n_components = K, covariance_type = 'full', tol = 1e-3, \
                          max_iter = 200, n_init = 3, init_params = 'kmeans', verbose = 0)
    
    # drawing a random subset of the whole training to use for training of this ensemble member
    # the remaining data points will also be classified
    indtemp = np.arange(0, ntr)
    Xtrs, Xtrr, Lontrs, Lontrr, Lattrs, Lattrr, indtrs, indtrr = train_test_split(Xtr, Lontr, Lattr, indtemp, test_size = (1-train_size))
    ntrs = np.shape(Xtrs)[0]
    ntrr = np.shape(Xtrr)[0]
    
    # fitting the model
    gmm.fit(Xtrs)
    
    # predicting the class for each grid point
    ytrs = gmm.predict(Xtrs)
    ytrr = gmm.predict(Xtrr)
    yte = gmm.predict(Xte)
    
    # predicting the probability for each point
    temp = gmm.predict_proba(Xtrs)
    probatrs = np.amax(temp, axis = 1)
    temp = gmm.predict_proba(Xtrr)
    probatrr = np.amax(temp, axis = 1)
    temp = gmm.predict_proba(Xte)
    probate = np.amax(temp, axis = 1)
    
    # saving the class means
    class_means[:, :, i] = gmm.means_
    
    # combining this ensemble member's train and remaining set
    ytr = np.empty((ntr), dtype = np.int16)
    ytr[indtrs] = ytrs
    ytr[indtrr] = ytrr
    probatr = np.empty((ntr))
    probatr[indtrs] = probatrs
    probatr[indtrr] = probatrr
    
    # combining the train and test set again
    y[indtr, i] = ytr
    y[indte, i] = yte
    proba[indtr, i] = probatr
    proba[indte, i] = probate
    
    # printing the ensemble number
    if ((N-i-1) % 25 == 0):
        print('%4i' % (N-i))
    else:
        print('%4i' % (N-i), end = ' ')

 200  199  198  197  196  195  194  193  192  191  190  189  188  187  186  185  184  183  182  181  180  179  178  177  176
 175  174  173  172  171  170  169  168  167  166  165  164  163  162  161  160  159  158  157  156  155  154  153  152  151
 150  149  148  147  146  145  144  143  142  141  140  139  138  137  136  135  134  133  132  131  130  129  128  127  126
 125  124  123  122  121  120  119  118  117  116  115  114  113  112  111  110  109  108  107  106  105  104  103  102  101
 100   99   98   97   96   95   94   93   92   91   90   89   88   87   86   85   84   83   82   81   80   79   78   77   76
  75   74   73   72   71   70   69   68   67   66   65   64   63   62   61   60   59   58   57   56   55   54   53   52   51
  50   49   48   47   46   45   44   43   42   41   40   39   38   37   36   35   34   33   32   31   30   29   28   27   26
  25   24   23   22   21   20   19   18   17   16   15   14   13   12   11   10    9    8    7    6    5    4    3    2    1


### Combining the results from the ensemble

In [7]:
def getSizeOfNestedList(listOfElem):
    ''' Get number of elements in a nested list'''
    count = 0
    # Iterate over the list
    for elem in listOfElem:
        # Check if type of element is list
        if type(elem) == list:  
            # Again call this function to get the size of this element
            count += getSizeOfNestedList(elem)
        else:
            count += 1    
    return count

In [8]:
# finding the list of all classes that appear in any of the models (classes with corr >= corrlim are considered same)
# the first N classes are from the 1st ensemble member
classes_all = np.expand_dims(class_means[0, :, 0], 0)
classes_matching = np.empty((K, N), dtype = np.int64)
classes_matching[0, 0] = 0
class_means_new = np.expand_dims(class_means[0, :, 0], (0, 1)).tolist()

num = getSizeOfNestedList(class_means_new)

# comparing the class means from the other ensemble members with the growing list of separate classes
for i in range(0, N):
    for j in range(K):
        
        if ((i == 0) and (j == 0)):
            continue
        
        r = np.empty((np.shape(classes_all)[0]))
        
        for k in range(len(r)):
            r[k] = pearsonr(class_means[j, :, i], classes_all[k, :])[0]
            
            
        if (np.amax(r) >= corrlim):
            temp = np.argmax(r)
            classes_matching[j, i] = temp
            class_means_new[temp].append(class_means[j, :, i].tolist())
        else:
            classes_all = np.concatenate((classes_all, np.reshape(class_means[j, :, i], (1, npcs))))
            classes_matching[j, i] = np.shape(classes_all)[0]-1
            class_means_new.append(np.expand_dims(class_means[j, :, i], 0).tolist())
    
    # finding and printing the number of class means after each ensemble member is sorted
    temp = num
    num = getSizeOfNestedList(class_means_new)
    #print('%2i -> %4i | %3i' % (i, num, (num-temp)/8))
            
tot_class_num = np.shape(classes_all)[0]

In [9]:
# assigning new class numbers to the results
yNew = np.empty((ntr+nte, N), dtype = np.int64)

for i in range(N):    
    for j in range(K):
        ind = np.squeeze(np.nonzero(y[:, i] == j))
        yNew[ind, i] = classes_matching[j, i]

In [10]:
# majority voting + likelihood (probability cutoff + weighted votes + removing small classes)
def soft_vote(x, proba, nan = -99, minproba = 0.5, minclass = 100):
    # array with samples as rows and ensemble models as columns
    
    # maority voting for the whole array
    x2 = np.empty((np.shape(x)[0]), dtype = np.int16)
    like = np.empty((np.shape(x)[0]))
    votes = []
    
    for i in range(np.shape(x)[0]):
        mask = np.nonzero((x[i, :] == nan) & (proba[i, :] < minproba))        
        classes = np.delete(x[i, :], mask) # removing empty and too weak votes
        weights = np.delete(proba[i, :], mask)
        votes.append(np.bincount(classes, weights = weights)) # number of votes for each class
        
        if len(votes[i]) == 0:
            x2[i] = -1
            like[i] = 0
        else:
            x2[i] = votes[i].argmax()       # class with the highest vote
            like[i] = votes[i].max()/np.shape(x)[1] # how many voted for this class
            
    # check if some of the classes are too small - if yes, use the next best not-too-small class for the grid points
    # that belong to the small classes
    
    # finding small classes
    maxclass = np.max(x)
    class_size = np.empty((maxclass+1), dtype = np.int32)
    for i in range(maxclass + 1):
        class_size[i] = len(np.nonzero(x2 == i)[0])
        
    # small classes
    small_classes = np.nonzero(class_size <= minclass)[0]
    
    # finding a new class for each of the grid points belonging to a small class
    for i in range(np.shape(x2)[0]):
        if np.isin(x2[i], small_classes, assume_unique = True):     #x2[i] in small_classes:
            # deleting the votes that blong to the small classes (temp = votes from only large enough classes)
            temp = np.zeros((len(votes[i])), dtype = np.int16)
            for k in range(len(temp)):
                if not np.isin(k, small_classes, assume_unique = True):  # k not in small_classes:
                    temp[k] = votes[i][k]
            
            # choosing the second best vote (or -1 if there are no other votes)
            #print((i, x2[i]), end = ' -> ')
            if np.any(temp):  # if any elements is not zero
                x2[i] = temp.argmax()
            else:
                x2[i] = -1
            #print(x2[i])
            like[i] = temp.max()/np.shape(x)[1]
            
    return [x2, like]

In [11]:
# majority voting
ySV, like = soft_vote(yNew, proba, nan = -99, minproba = minproba, minclass = minsize)

In [12]:
# how many grid points exist in each class
points_per_class = np.empty((tot_class_num))
for i in range(tot_class_num):
    points_per_class[i] = len(np.squeeze(np.nonzero(ySV == i)))
    print('class %2i: %8i' % (i, points_per_class[i]))

class  0:      319
class  1:        0
class  2:      313
class  3:      315
class  4:      287
class  5:      430
class  6:      717
class  7:        0
class  8:        0


In [13]:
# dropping the empty classes
classes = np.arange(tot_class_num)
ind = np.squeeze(points_per_class > 0)
classes = classes[ind]
new_class_num = len(classes)
classes_new = np.squeeze(classes_all[ind, :])
class_means_new = [a for (a, m) in zip(class_means_new, ind) if m]
print('Total number of classes: %2i' % tot_class_num)
print('Final number of classes: %2i' % new_class_num)

Total number of classes:  9
Final number of classes:  6


In [14]:
# changing the class numbers to be from 1 to new_class_num (before the classes started at 0!)
for i in range(new_class_num):
    ySV[ySV == classes[i]] = i + 1001
ySV = ySV - 1000

In [15]:
# how many grid points exist in each class
points_per_class = np.empty((new_class_num))
for i in range(new_class_num):
    points_per_class[i] = len(np.squeeze(np.nonzero(ySV == i+1)))
    print('class %2i: %8i' % (i+1, points_per_class[i]))

class  1:      319
class  2:      313
class  3:      315
class  4:      287
class  5:      430
class  6:      717


### Saving the results

In [16]:
# reshape the data back to a grid
res_grid = np.nan * np.ones((nlat*nlon), dtype = np.int16)
res_grid[ind_ocean] = np.squeeze(ySV)
res_grid = np.reshape(res_grid, (nlat, nlon),'C')

like_grid = np.nan * np.ones((nlat*nlon))
like_grid[ind_ocean] = np.squeeze(like)
like_grid = np.reshape(like_grid, (nlat, nlon), 'C')

In [17]:
# class means from voting (special treatment because it is not a regular np array that can be converted to matlab array)
ensemble_means = dict()

# creating a dictionary with class number as key and a numpy aray of shape
#     ensemble members that have this class x number of PCs
# as values
for i in range(new_class_num):
    ensemble_means[str(i)] = np.array(class_means_new[i])
    
# saving the dictionary into a matlab readable file
scipy.io.savemat(res+modelname+'ensemble_class_means.mat', ensemble_means)

# all other regular variables
# Lontr, Lattr = coordinates of the training set grid points
# Lonte, Latte = coordinates of the test set grid points
# lon, lat = coordinates in vector format
# Lon2, Lat2, = coordinates of all results (combined train and test) in an array format
# res_grid = classes for each grid point on a grid
# like_grid = likelihood for each grid point on a grid
# X = input data
# ySV = results in array format
scipy.io.savemat(res+modelname+'.mat', dict(Lontr = Lontr, Lattr = Lattr, \
                                            Lonte = Lonte, Latte = Latte, \
                                            lon = lon, lat = lat, \
                                            Lon = Lon2, Lat = Lat2, \
                                            res_grid = res_grid, \
                                            like_grid = like_grid, \
                                            X = X, \
                                            y = ySV))